In [ ]:
# Library import
import pandas as pd
import numpy as np
import regex as re
import os

In [ ]:
#import dataset
cvd_df = pd.read_csv("Datasets/mimic-iv-ext-cardiac-disease-1.0.0/heart_diagnoses.csv")
cvd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4761 entries, 0 to 4760
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   note_id          4761 non-null   object
 1   subject_id       4761 non-null   int64 
 2   hadm_id          4761 non-null   int64 
 3   note_type        4761 non-null   object
 4   note_seq         4761 non-null   int64 
 5   charttime        4761 non-null   object
 6   storetime        4761 non-null   object
 7   HPI              4761 non-null   object
 8   physical_exam    4761 non-null   object
 9   chief_complaint  4749 non-null   object
 10  invasions        4749 non-null   object
 11  X-ray            4761 non-null   object
 12  CT               4761 non-null   object
 13  Ultrasound       4761 non-null   object
 14  CATH             4761 non-null   object
 15  ECG              4761 non-null   object
 16  MRI              4761 non-null   object
 17  reports          4761 non-null   

In [3]:
cvd_df.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,HPI,physical_exam,chief_complaint,invasions,X-ray,CT,Ultrasound,CATH,ECG,MRI,reports
0,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,\nNone\n \n,"['___:', 'views of the chest demonstrate low l...",[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...
1,10000980-DS-21,10000980,26913865,DS,21,2189-07-03 00:00:00,2189-07-03 19:50:00,":\nThis is a ___ M with history of diabetes, d...",ADMISSION EXAM:\nGeneral- appears comfortable...,\ndyspnea\n \n,\nCardiac catheterization ___\n\n \n,[': ___\nRight upper lobe pneumonia or mass. ...,['CHEST: ___\n1. Diffuse confluent ground-gla...,[': ___\nThe left atrium is elongated. There i...,[': ___\n1. Selective coronary angiography of ...,[],[],Sinus bradycardia with sinus arrhythmia | Prol...
2,10002013-DS-8,10002013,24760295,DS,8,2160-07-12 00:00:00,2160-07-14 13:59:00,":\n___ w/ PMH of CAD s/p PCI x3, s/p off-pump ...",Admission:\nVS- T 99.4 BP 157/88 HR 118 RR 24 ...,\nchest pain\n \n,\ncardiac catheterization\n\n \n,['___- New moderate left pleural effusion with...,['Chest ___. No CT evidence for pulmonary emb...,[],['___. Selective coronary angiography of this ...,['on admission- Sinus tachycardia. Extensive S...,[],Sinus tachycardia | Extensive ST-T changes may...
3,10002155-DS-8,10002155,23822395,DS,8,2129-08-18 00:00:00,2129-08-19 12:29:00,:\n___ is a ___ yo female with a past medical ...,"GENERAL: WDWN in NAD. Oriented x3. Mood, affec...",\nchest pressure\n \n,\nCardiac cathetherization\n\n \n,[],"['anteriorly, no \ncrackles, wheezes or rhonch...",[],[': \nLMCA: 40% distal\nLAD: Occluded difficu...,[': new LBBB with STE in V1 -V3 & V5 that in ...,[],*** CONSIDER ACUTE ST ELEVATION MI *** | Sinus...
4,10004457-DS-10,10004457,28723315,DS,10,2141-08-13 00:00:00,2141-08-13 18:50:00,:\nMr. ___ is a ___ with a hx of CAD (s/p DES ...,On Admission:\nVS- 97.8 157/64 101 18 98% RA \...,"\nAbnormal Stress Test, New AI\n \n",\nNone\n \n,[],['/MR or TEE is suggested to \nevaluate for po...,[': \nAnginal type symptoms with borderline i...,[],['(see exercise report for details). Resting s...,[],Sinus rhythm | Normal ECG


In [4]:
diagnoses = pd.read_csv("Datasets/mimic-iv-ext-cardiac-disease-1.0.0/heart_diagnoses_all.csv")
diagnoses.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61384 entries, 0 to 61383
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subject_id  61384 non-null  int64 
 1   hadm_id     61384 non-null  int64 
 2   seq_num     61384 non-null  int64 
 3   icd_code    61384 non-null  object
 4   long_title  61384 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.3+ MB


In [5]:
diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,long_title
0,10000980,26913865,1,I21,Acute myocardial infarction
1,10000980,26913865,2,I50,Heart failure
2,10000980,26913865,3,I25,Chronic ischemic heart disease
3,10000980,26913865,4,N18,Chronic kidney disease (CKD)
4,10000980,26913865,5,I34,Nonrheumatic mitral valve disorders


In [6]:
diagnoses['hadm_id'].value_counts()

hadm_id
22133572    37
25891316    36
28790642    35
26037283    35
27112917    34
            ..
26527179     1
28483693     1
22011835     1
25360274     1
24563659     1
Name: count, Length: 4761, dtype: int64

In [7]:
diagnoses['subject_id'].value_counts()

subject_id
12426774    87
19270175    82
16439884    73
12143980    73
11296936    71
            ..
19277239     1
16020649     1
14864163     1
15467340     1
17331657     1
Name: count, Length: 4289, dtype: int64

In [8]:
# Filter all rows in diagnoses where hadm_id is duplicated
duplicated_hadm_ids = diagnoses['hadm_id'][diagnoses['hadm_id'].duplicated(keep=False)]
diagnoses_duplicated = diagnoses[diagnoses['hadm_id'].isin(duplicated_hadm_ids)]
diagnoses_duplicated.head()

,subject_id,hadm_id,seq_num,icd_code,long_title
0,10000980,26913865,1,I21,Acute myocardial infarction
1,10000980,26913865,2,I50,Heart failure
2,10000980,26913865,3,I25,Chronic ischemic heart disease
3,10000980,26913865,4,N18,Chronic kidney disease (CKD)
4,10000980,26913865,5,I34,Nonrheumatic mitral valve disorders


In [9]:
# Merhge the two dataframes
cvd_merged = pd.merge(cvd_df, diagnoses, on="hadm_id", how="left")
cvd_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61384 entries, 0 to 61383
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   note_id          61384 non-null  object
 1   subject_id_x     61384 non-null  int64 
 2   hadm_id          61384 non-null  int64 
 3   note_type        61384 non-null  object
 4   note_seq         61384 non-null  int64 
 5   charttime        61384 non-null  object
 6   storetime        61384 non-null  object
 7   HPI              61384 non-null  object
 8   physical_exam    61384 non-null  object
 9   chief_complaint  61224 non-null  object
 10  invasions        61224 non-null  object
 11  X-ray            61384 non-null  object
 12  CT               61384 non-null  object
 13  Ultrasound       61384 non-null  object
 14  CATH             61384 non-null  object
 15  ECG              61384 non-null  object
 16  MRI              61384 non-null  object
 17  reports          61384 non-null

In [10]:
cvd_merged.head(5)

,note_id,subject_id_x,hadm_id,note_type,note_seq,charttime,storetime,HPI,physical_exam,chief_complaint,...,CT,Ultrasound,CATH,ECG,MRI,reports,subject_id_y,seq_num,icd_code,long_title
0,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,1,I50,Heart failure
1,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,2,I24,Other acute ischemic heart diseases
2,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,3,I12,Hypertensive chronic kidney disease
3,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,4,E78,Disorders of lipoprotein metabolism and other ...
4,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,[],['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,5,E11,Type 2 diabetes mellitus


### Filtered the merged dafaframes to focus on primary diagnosis of AMI 

In [12]:

cvd_primDiag = cvd_merged[(cvd_merged['seq_num'] == 1)].reset_index(drop=True)
cvd_primDiag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4761 entries, 0 to 4760
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   note_id          4761 non-null   object
 1   subject_id_x     4761 non-null   int64 
 2   hadm_id          4761 non-null   int64 
 3   note_type        4761 non-null   object
 4   note_seq         4761 non-null   int64 
 5   charttime        4761 non-null   object
 6   storetime        4761 non-null   object
 7   HPI              4761 non-null   object
 8   physical_exam    4761 non-null   object
 9   chief_complaint  4749 non-null   object
 10  invasions        4749 non-null   object
 11  X-ray            4761 non-null   object
 12  CT               4761 non-null   object
 13  Ultrasound       4761 non-null   object
 14  CATH             4761 non-null   object
 15  ECG              4761 non-null   object
 16  MRI              4761 non-null   object
 17  reports          4761 non-null   

In [13]:
cvd_primDiag["long_title"].value_counts()

long_title
Heart failure                                                                          1420
Acute myocardial infarction                                                            1398
Chronic ischemic heart disease                                                          633
Atrial fibrillation and flutter                                                         434
Nonrheumatic aortic valve disorders                                                     229
Other diseases of pericardium                                                           146
Paroxysmal tachycardia                                                                  128
Acute pericarditis                                                                       81
Atrioventricular and left bundle-branch block                                            67
Nonrheumatic mitral valve disorders                                                      49
Acute and subacute endocarditis                                      

In [14]:
cvd_primDiag['AMI label'] = (cvd_primDiag["icd_code"] == "I21").map({True: "Yes", False: "No"})
display(cvd_primDiag["AMI label"].value_counts())
cvd_primDiag.head()

AMI label
No     3363
Yes    1398
Name: count, dtype: int64

,note_id,subject_id_x,hadm_id,note_type,note_seq,charttime,storetime,HPI,physical_exam,chief_complaint,...,Ultrasound,CATH,ECG,MRI,reports,subject_id_y,seq_num,icd_code,long_title,AMI label
0,10000980-DS-20,10000980,29654838,DS,20,2188-01-05 00:00:00,2188-01-06 20:49:00,":\n___ yo woman with h/o hypertension, hyperli...","Admission exam:\nGENERAL- Oriented x3. Mood, a...",\nShortness of breath\n \n,...,['___:\nThe left atrium is mildly dilated. The...,[],['___ 7:56:06 ___ \nBaseline artifact. Sinus...,[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,1,I50,Heart failure,No
1,10000980-DS-21,10000980,26913865,DS,21,2189-07-03 00:00:00,2189-07-03 19:50:00,":\nThis is a ___ M with history of diabetes, d...",ADMISSION EXAM:\nGeneral- appears comfortable...,\ndyspnea\n \n,...,[': ___\nThe left atrium is elongated. There i...,[': ___\n1. Selective coronary angiography of ...,[],[],Sinus bradycardia with sinus arrhythmia | Prol...,10000980,1,I21,Acute myocardial infarction,Yes
2,10002013-DS-8,10002013,24760295,DS,8,2160-07-12 00:00:00,2160-07-14 13:59:00,":\n___ w/ PMH of CAD s/p PCI x3, s/p off-pump ...",Admission:\nVS- T 99.4 BP 157/88 HR 118 RR 24 ...,\nchest pain\n \n,...,[],['___. Selective coronary angiography of this ...,['on admission- Sinus tachycardia. Extensive S...,[],Sinus tachycardia | Extensive ST-T changes may...,10002013,1,I21,Acute myocardial infarction,Yes
3,10002155-DS-8,10002155,23822395,DS,8,2129-08-18 00:00:00,2129-08-19 12:29:00,:\n___ is a ___ yo female with a past medical ...,"GENERAL: WDWN in NAD. Oriented x3. Mood, affec...",\nchest pressure\n \n,...,[],[': \nLMCA: 40% distal\nLAD: Occluded difficu...,[': new LBBB with STE in V1 -V3 & V5 that in ...,[],*** CONSIDER ACUTE ST ELEVATION MI *** | Sinus...,10002155,1,I21,Acute myocardial infarction,Yes
4,10004457-DS-10,10004457,28723315,DS,10,2141-08-13 00:00:00,2141-08-13 18:50:00,:\nMr. ___ is a ___ with a hx of CAD (s/p DES ...,On Admission:\nVS- 97.8 157/64 101 18 98% RA \...,"\nAbnormal Stress Test, New AI\n \n",...,[': \nAnginal type symptoms with borderline i...,[],['(see exercise report for details). Resting s...,[],Sinus rhythm | Normal ECG,10004457,1,I35,Nonrheumatic aortic valve disorders,No


# Removing Special Characters to normalize data

In [15]:
## gt a list of all special charactyers in tthe text data
combineText = cvd_primDiag.select_dtypes(include="object")

allCombined_text = " ".join(combineText.fillna("").astype(str).values.ravel())

special_chars = set(re.findall(r"[^\w\s]", allCombined_text))

special_chars = sorted(special_chars)
print(special_chars)


['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '`', '{', '|', '}', '~', '\x80', '\x82', '\x83', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x99', '\x9a', '\x9d', '¢', '©', '¬', '°', '²', '×']


In [16]:
# cleaning text in each column expect for one that need special characters
def clean_text(x):
    if not isinstance(x, str):
        return x
    x = re.sub(r"\\n|\n", " ", x)
    x = re.sub(r"[?!\"#\$&\*\+\^@|<>\{\}\[\]\(\)\'=\\~=`\x80-\x99©²\-]", " ", str(x).strip("[]':-\n"))
    x = re.sub(r"_{2,}", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    x = re.sub(r"\."," ", x)
    x = re.sub(r"\:"," ", x)
    x = re.sub(r"\s+", " ", x)
    
    return x.strip(" :")

for col in cvd_primDiag.columns:
    if col in ["note_id" ,"charttime","storetime"]: #igone columns
        continue

    if cvd_primDiag[col].dtype == "object":
        cvd_primDiag[col] = cvd_primDiag[col].apply(clean_text)


# Removing Duplicates

In [18]:
# columns dtype conversion 
cvd_primDiag["storetime"] = pd.to_datetime(cvd_primDiag["storetime"], errors="coerce")
cvd_primDiag["charttime"] = pd.to_datetime(cvd_primDiag["charttime"], errors="coerce")
cvd_primDiag = cvd_primDiag.drop(columns=['subject_id_y'])


# Checking for duplicated memeber ids 
cvd_primDiag[cvd_primDiag.duplicated(subset=['subject_id_x'], keep=False)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 863 entries, 0 to 4756
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   note_id          863 non-null    object        
 1   subject_id_x     863 non-null    int64         
 2   hadm_id          863 non-null    int64         
 3   note_type        863 non-null    object        
 4   note_seq         863 non-null    int64         
 5   charttime        863 non-null    datetime64[ns]
 6   storetime        863 non-null    datetime64[ns]
 7   HPI              863 non-null    object        
 8   physical_exam    863 non-null    object        
 9   chief_complaint  860 non-null    object        
 10  invasions        860 non-null    object        
 11  X-ray            863 non-null    object        
 12  CT               863 non-null    object        
 13  Ultrasound       863 non-null    object        
 14  CATH             863 non-null    object       

In [19]:
# Filters the first note per patient i.e. firts visit
cvd_primDiag_first = cvd_primDiag.sort_values(by=["subject_id_x", "storetime"])
cvd_primDiag_first = cvd_primDiag_first.drop_duplicates(subset="subject_id_x", keep="first")
cvd_primDiag_first.head()

,note_id,subject_id_x,hadm_id,note_type,note_seq,charttime,storetime,HPI,physical_exam,chief_complaint,...,CT,Ultrasound,CATH,ECG,MRI,reports,seq_num,icd_code,long_title,AMI label
0,10000980-DS-20,10000980,29654838,DS,20,2188-01-05,2188-01-06 20:49:00,"yo woman with h/o hypertension, hyperlipidemia...","Admission exam GENERAL Oriented x3 Mood, affec...",Shortness of breath,...,,The left atrium is mildly dilated The estimate...,,7 56 06 Baseline artifact Sinus rhythm The Q T...,,Sinus bradycardia with sinus arrhythmia Prolon...,1,I50,Heart failure,No
2,10002013-DS-8,10002013,24760295,DS,8,2160-07-12,2160-07-14 13:59:00,"w/ PMH of CAD s/p PCI x3, s/p off pump CABG x3...",Admission VS T 99 4 BP 157/88 HR 118 RR 24 96%...,chest pain,...,Chest No CT evidence for pulmonary embolus 2 S...,,Selective coronary angiography of this right d...,on admission Sinus tachycardia Extensive ST se...,,Sinus tachycardia Extensive ST T changes may b...,1,I21,Acute myocardial infarction,Yes
3,10002155-DS-8,10002155,23822395,DS,8,2129-08-18,2129-08-19 12:29:00,is a yo female with a past medical history of ...,"GENERAL WDWN in NAD Oriented x3 Mood, affect a...",chest pressure,...,"anteriorly, no crackles, wheezes or rhonchi AB...",,LMCA 40% distal LAD Occluded difficult to cros...,new LBBB with STE in V1 V3 V5 that in some lea...,,CONSIDER ACUTE ST ELEVATION MI Sinus arrhythmi...,1,I21,Acute myocardial infarction,Yes
4,10004457-DS-10,10004457,28723315,DS,10,2141-08-13,2141-08-13 18:50:00,Mr is a with a hx of CAD s/p DES to LAD in who...,On Admission VS 97 8 157/64 101 18 98% RA GENE...,"Abnormal Stress Test, New AI",...,/MR or TEE is suggested to evaluate for possib...,Anginal type symptoms with borderline ischemic...,,see exercise report for details Resting systol...,,Sinus rhythm Normal ECG,1,I35,Nonrheumatic aortic valve disorders,No
5,10007058-DS-2,10007058,22954658,DS,2,2167-11-11,2167-11-12 11:39:00,Mr is a healthy year old male who presented wi...,"ADMISSION PHYSICAL EXAM VS T 98 2, HR 63, BP 1...",Chest pain,...,1 Infrarenal abdominal aortic aneurysm as deta...,The left atrium is mildly dilated No atrial se...,RCA occluded with thrombus in mid to distal ve...,"NSR, nl axis, no ST changes",,"Age not entered, assumed to be 50 years old fo...",1,I21,Acute myocardial infarction,Yes


In [20]:
import textwrap

for x in range(5):
    text = cvd_primDiag_first[cvd_primDiag_first.duplicated(subset=['physical_exam'], keep=False)].reset_index(drop=True).loc[x, "physical_exam"]
    wrapped_text = "\n".join(textwrap.wrap(text, width=80))
    print(wrapped_text)
    print("************************")

GENERAL WDWN in NAD Oriented x3 Mood, affect appropriate HEENT NCAT Sclera
anicteric PERRL, EOMI Conjunctiva were pink, no pallor or cyanosis of the oral
mucosa No xanthalesma
************************
ADMISSION PHYSICAL EXAMINATION
************************
ADMISSION EXAM GENERAL NAD Oriented x3 Mood, affect appropriate HEENT NCAT
Sclera anicteric PERRL, EOMI Conjunctiva were pink, no pallor or cyanosis of the
oral mucosa No xanthalesma
************************
PHYSICAL EXAM ON ADMISSION
************************
GENERAL NAD Oriented x3 Mood, affect appropriate HEENT NCAT Sclera anicteric
PERRL, EOMI Conjunctiva were pink, no pallor or cyanosis of the oral mucosa No
xanthalesma
************************


In [21]:
cvd_primDiag_first = cvd_primDiag_first.applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)

/var/tmp/ipykernel_67174/35410627.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cvd_primDiag_first = cvd_primDiag_first.applymap(lambda x: None if isinstance(x, str) and x.strip() == "" else x)


In [22]:
# Filling null spaces with nothing since there are no notes (might drop them )
text_cols = ["HPI", "physical_exam", "chief_complaint", "reports"]
cvd_primDiag_first[text_cols] = cvd_primDiag_first[text_cols].fillna("")


for col in ["X-ray", "CT", "MRI", "Ultrasound", "CATH", "ECG","invasions"]:
    # cvd_primDiag_first[f"{col}_missing"] = cvd_primDiag_first[col].isna().astype(int)
    cvd_primDiag_first[col] = cvd_primDiag_first[col].fillna("not performed")


# Class Balnacing

In [23]:
cvd_primDiag_first["AMI label"].value_counts()

AMI label
No     2985
Yes    1304
Name: count, dtype: int64

In [24]:
from sklearn.utils import resample

AMI = cvd_primDiag_first[cvd_primDiag_first["AMI label"] == "Yes"]
no_AMI = cvd_primDiag_first[cvd_primDiag_first["AMI label"] == "No"]

min_AMI = min(len(AMI), len(no_AMI))

no_AMI_new = resample(no_AMI, replace= False, n_samples= min_AMI , random_state= 42)

cvd_primDiag_first_Bal = pd.concat([AMI , no_AMI_new]).sample(frac=1, random_state=42).reset_index(drop = True)

In [25]:
cvd_primDiag_first_Bal.head()

,note_id,subject_id_x,hadm_id,note_type,note_seq,charttime,storetime,HPI,physical_exam,chief_complaint,...,CT,Ultrasound,CATH,ECG,MRI,reports,seq_num,icd_code,long_title,AMI label
0,13847595-DS-18,13847595,22624533,DS,18,2116-09-01,2116-09-04 15:45:00,y o with a history of dizziness and palpitatio...,Admission PE VS HR 66 RR 16 BP 115/80 O2 sat 9...,"syncope x 1, ongoing palpitations, dizziness, ...",...,not performed,guided access of right IJ sheath placed in rig...,Impressions The benefits and risks of the proc...,not performed,not performed,Sinus arrhythmia with 1st degree A V block Dem...,1,I47,Paroxysmal tachycardia,No
1,10834692-DS-19,10834692,28985445,DS,19,2111-11-14,2111-11-15 13:19:00,"year female with h/o DMT2, OSA, Asthma, and Dy...",VS T 97 6 BP 160/70 HR 66RR 20 O2 sat 97% 2LNC...,shortness of breath,...,not performed,The left atrium is dilated Left ventricular wa...,COMMENTS 1 Limited selective coronary angiogra...,not performed,not performed,Sinus rhythm Normal ECG Sinus rhythm report ma...,1,I25,Chronic ischemic heart disease,No
2,11652381-DS-6,11652381,26132159,DS,6,2148-12-17,2148-12-17 18:53:00,"PMHx CAD s/p BMS to D1 or , hx s CMP reportedl...",Admission Exam VS T 97 6 BP 118/95 HR 85 RR 20...,"chest pain, shortness of breath",...,is recommended to further assess Abdominal,1 Gallbladder is decompressed No gallstones ar...,not performed,not performed,not performed,Sinus rhythm Possible inferior infarct age und...,1,I21,Acute myocardial infarction,Yes
3,15847483-DS-17,15847483,20867307,DS,17,2142-07-14,2142-07-14 13:14:00,"HPI This is a year old male hx ETOH abuse, chr...","On Admission VS Temperature 98 3 Fahrenheit, b...",Chest pain Abdominal pain,...,A/P 1 Moderate intrahepatic and extrahepatic b...,The left atrium is mildly dilated No atrial se...,indications chest pain Summary 1 A percutaneou...,not performed,not performed,Sinus rhythm Probable left atrial enlargement ...,1,I25,Chronic ischemic heart disease,No
4,18177277-DS-13,18177277,23302028,DS,13,2155-10-03,2155-10-04 19:17:00,"Ms is an year old lady, with a PMHx of CAD s/p...","ADMISSION PHYSICAL VS 98 3, 132/75, 58, 16, 98...",Chest Pain,...,not performed,The left atrium is moderately dilated There is...,"PRELIM PCW 13, LVED 14 AoSt and MS not severe ...",not performed,not performed,Sinus bradycardia Left axis deviation Poor R w...,1,I25,Chronic ischemic heart disease,No


In [26]:
cvd_primDiag_first_Bal["AMI label"].value_counts()

AMI label
No     1304
Yes    1304
Name: count, dtype: int64

# Creating promts for prediction using the COT aproache

In [27]:
row = cvd_primDiag_first_Bal.loc[0]
for col, value in row.items():
    print(f"{col}: {value}")

note_id: 13847595-DS-18
subject_id_x: 13847595
hadm_id: 22624533
note_type: DS
note_seq: 18
charttime: 2116-09-01 00:00:00
storetime: 2116-09-04 15:45:00
HPI: y o with a history of dizziness and palpitations beginning in He reports he awakened to get up to use bathroom one night in after being asleep for a short time; he remembers walking into the bathroom and passed out No reported dizziness, lightheadedness or chest pain Wife witnessed event He woke up instantaneously and knew he had passed out He contacted his PCP the next day, was seen the following week and reported ongoing palpitations with associated dizziness He was referred to cardiology Evaluated at ; echo stress demonstrating transient heart block He reports his resting HR at the beginning of the test was 137 bpm He had a Dual placed on He was then noted to be in sinus tachycardia with reported dizziness and a beta blocker was started A Holter demonstrated a several hour run of long RP tachycardia He was offered Flecainide o

In [28]:
def CoT_zeroshot(row):
    cc = row.get("chief_complaint", "").strip()
    hpi = row.get("HPI", "").strip()
    pe = row.get("physical_exam", "").strip()
    ecg = row.get("ECG", "").strip()
    cath = row.get("CATH", "").strip()
    xray = row.get("X-ray", "").strip()
    us = row.get("Ultrasound", "").strip()
    

    return f""" A patient with the following symptoms
Chief Complaint: {cc}
History of Present Illness: {hpi}
Physical Exam: {pe}
ECG Findings: {ecg}
Catheterization Report: {cath}
X-ray Findings: {xray}
Ultrasound: {us}

Final Thought:
Evaluate the likelihood of an acute myocardial infarction based on the findings.

Reason step-by-step, then answer YES or NO.

"""

def CoT_fewshot(row):
    cc = row.get("chief_complaint", "").strip()
    hpi = row.get("HPI", "").strip()
    pe = row.get("physical_exam", "").strip()
    ecg = row.get("ECG", "").strip()
    cath = row.get("CATH", "").strip()
    xray = row.get("X-ray", "").strip()
    us = row.get("Ultrasound", "").strip()


    return f"""
Example 1:
A patient with the following symptoms
Chief Complaint: STEMI
ECG: ST elevations in inferior leads, conduction defect, IVCD.
CATH: Severe 3-vessel CAD, SVG to RPDA with 65% distal lesion, successful angioplasty.
X-ray: Post-CABG status, no pulmonary edema.
Ultrasound: LVEF ~40%, RV dilation, moderate MR/TR, inferolateral akinesis.
Reports: Inferior STEMI, cardiogenic shock, VT requiring defib.

Final Thought:
Step-by-step:

Patient has a known history of CAD and recent CABG.

He presents with syncope, wide-complex tachycardia, and hypotension — signs of instability.

ECG shows inferior ST elevations (II, aVF) with reciprocal depressions.

Labs show elevated troponin (1.5), CKMB 12.4, consistent with myocardial necrosis.

Cardiac cath confirms flow-limiting lesion and successful balloon angioplasty.

Imaging (Echo) supports ischemic changes and reduced LVEF.
Findings strongly support an acute myocardial infarction diagnosis.

Answer: YES

Now evaluate the following case:
Chief Complaint: {cc}
History of Present Illness: {hpi}
Physical Exam: {pe}
ECG Findings: {ecg}
Catheterization Report: {cath}
X-ray Findings: {xray}
Ultrasound: {us}

Final Thought:
Evaluate the likelihood of an acute myocardial infarction based on the findings.

Reason step-by-step, then answer YES or NO.
"""


# Apply it to your DataFrame
cvd_primDiag_first_Bal["zs_prompt"] = cvd_primDiag_first_Bal.apply(CoT_zeroshot, axis=1)
cvd_primDiag_first_Bal["fs_prompt"] = cvd_primDiag_first_Bal.apply(CoT_fewshot, axis=1)


In [29]:
cvd_primDiag_first_Bal.to_csv("Datasets/mimic_iv_ami.csv", index = False)

In [30]:
cvd_primDiag_first_Bal.loc[0, "zs_prompt"]

' A patient with the following symptoms\nChief Complaint: syncope x 1, ongoing palpitations, dizziness, and DOE\nHistory of Present Illness: y o with a history of dizziness and palpitations beginning in He reports he awakened to get up to use bathroom one night in after being asleep for a short time; he remembers walking into the bathroom and passed out No reported dizziness, lightheadedness or chest pain Wife witnessed event He woke up instantaneously and knew he had passed out He contacted his PCP the next day, was seen the following week and reported ongoing palpitations with associated dizziness He was referred to cardiology Evaluated at ; echo stress demonstrating transient heart block He reports his resting HR at the beginning of the test was 137 bpm He had a Dual placed on He was then noted to be in sinus tachycardia with reported dizziness and a beta blocker was started A Holter demonstrated a several hour run of long RP tachycardia He was offered Flecainide or ablation but dec

In [31]:
cvd_primDiag_first_Bal.loc[1, "fs_prompt"]

'\nExample 1:\nA patient with the following symptoms\nChief Complaint: STEMI\nECG: ST elevations in inferior leads, conduction defect, IVCD.\nCATH: Severe 3-vessel CAD, SVG to RPDA with 65% distal lesion, successful angioplasty.\nX-ray: Post-CABG status, no pulmonary edema.\nUltrasound: LVEF ~40%, RV dilation, moderate MR/TR, inferolateral akinesis.\nReports: Inferior STEMI, cardiogenic shock, VT requiring defib.\n\nFinal Thought:\nStep-by-step:\n\nPatient has a known history of CAD and recent CABG.\n\nHe presents with syncope, wide-complex tachycardia, and hypotension — signs of instability.\n\nECG shows inferior ST elevations (II, aVF) with reciprocal depressions.\n\nLabs show elevated troponin (1.5), CKMB 12.4, consistent with myocardial necrosis.\n\nCardiac cath confirms flow-limiting lesion and successful balloon angioplasty.\n\nImaging (Echo) supports ischemic changes and reduced LVEF.\nFindings strongly support an acute myocardial infarction diagnosis.\n\nAnswer: YES\n\nNow eval

In [32]:
from transformers import AutoTokenizer

# Load the tokenizer you're using
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Your full prompt
prompt = cvd_primDiag_first_Bal.loc[1, "zs_prompt"]

# Tokenize
tokens = tokenizer.tokenize(prompt)
print("Token count:", len(tokens))


/jet/home/oali2/.local/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Token count: 1236


In [33]:

cvd_primDiag_first_Bal["token_count"] = cvd_primDiag_first_Bal["fs_prompt"].apply(
    lambda prompt: len(tokenizer.tokenize(prompt))
)


print(cvd_primDiag_first_Bal[["fs_prompt", "token_count"]].head())



                                           fs_prompt  token_count
0  \nExample 1:\nA patient with the following sym...         1747
1  \nExample 1:\nA patient with the following sym...         1515
2  \nExample 1:\nA patient with the following sym...         1201
3  \nExample 1:\nA patient with the following sym...         2421
4  \nExample 1:\nA patient with the following sym...         1812


In [35]:
cvd_primDiag_first_Bal["token_count"].describe()

count    2608.000000
mean     1460.462807
std       367.392544
min       663.000000
25%      1207.000000
50%      1412.500000
75%      1658.250000
max      3693.000000
Name: token_count, dtype: float64